In [35]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [36]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [37]:
from azureml.core import Workspace
ws=Workspace.from_config()
ws

Workspace.create(name='data_work', subscription_id='882060cd-0ab4-42e7-9d6d-b23f4295c991', resource_group='data_resource')

In [38]:
import os

# Create a folder for the data files
data_folder = 'data'
os.makedirs(data_folder, exist_ok=True)
print(data_folder, 'folder created')

data folder created


In [39]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


In [40]:

pip install --upgrade azureml-sdk


Note: you may need to restart the kernel to use updated packages.


In [41]:
from azureml.core import Workspace, Dataset

In [42]:
#upload data to datastores
from azureml.core import Dataset
from azureml.data.datapath import DataPath
default_ds = ws.get_default_datastore()
Dataset.File.upload_directory(src_dir='data',
                              target=DataPath(default_ds, 'Users/divyapratapsingh/data/gld_price_data.csv')
                              )

Validating arguments.
Arguments validated.
'overwrite' is set to False. Any file already present in the target will be skipped.'
Uploading files from '/mnt/batch/tasks/shared/LS_root/mounts/clusters/ksquarescomp2/code/Users/divyapratapsingh/data' to 'Users/divyapratapsingh/data/gld_price_data.csv'
Copying 3 files with concurrency set to 2
Skipped /mnt/batch/tasks/shared/LS_root/mounts/clusters/ksquarescomp2/code/Users/divyapratapsingh/data/.amlignore, file 1 out of 3. Target already exists.
Skipped /mnt/batch/tasks/shared/LS_root/mounts/clusters/ksquarescomp2/code/Users/divyapratapsingh/data/.amlignore.amltmp, file 2 out of 3. Target already exists.
Skipped /mnt/batch/tasks/shared/LS_root/mounts/clusters/ksquarescomp2/code/Users/divyapratapsingh/data/gld_price_data.csv, file 3 out of 3. Target already exists.
Files copied=0, skipped=3, failed=0
Creating new dataset


{
  "source": [
    "('workspaceblobstore', '/Users/divyapratapsingh/data/gld_price_data.csv')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ]
}

In [43]:
from azureml.core import Dataset

# Get the default datastore
default_ds = ws.get_default_datastore()

#Create a tabular dataset from the path on the datastore (this may take a short while)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'Users/divyapratapsingh/data/gld_price_data.csv'))

# Display the first 20 rows as a Pandas dataframe
tab_data_set.take(20).to_pandas_dataframe()

/anaconda/envs/jupyter_env/lib/python3.8/site-packages/azureml/dataprep/api/_dataframereader.py:578: UserWarning: Using alternate reader. Inconsistent or mixed schemas detected across partitions: partition had different number of columns. The first partition has 1 columns. Found partition has 6 columns.
First partition columns (ordered): ['## This file was auto generated by the Azure Machine Learning Studio. Please do not remove.']
Found Partition has columns (ordered): ['## This file was auto generated by the Azure Machine Learning Studio. Please do not remove.', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6']
  warnings.warn("Using alternate reader. " + reason)
/anaconda/envs/jupyter_env/lib/python3.8/site-packages/azureml/dataprep/api/_dataframereader.py:461: UserWarning: Please install pyarrow>=0.16.0 for improved performance of to_pandas_dataframe. You can ensure the correct version is installed by running: pip install pyarrow>=0.16.0 --upgrade
  warnings.warn(


,## This file was auto generated by the Azure Machine Learning Studio. Please do not remove.,Column2,Column3,Column4,Column5,Column6
0,## Read more about the .amlignore file here: h...,NaN,NaN,NaN,NaN,NaN
1,,NaN,NaN,NaN,NaN,NaN
2,.ipynb_aml_checkpoints/,NaN,NaN,NaN,NaN,NaN
3,*.amltmp,NaN,NaN,NaN,NaN,NaN
4,*.amltemp,NaN,NaN,NaN,NaN,NaN
5,## Read more about the .amlignore file here: h...,NaN,NaN,NaN,NaN,NaN
6,,NaN,NaN,NaN,NaN,NaN
7,.ipynb_aml_checkpoints/,NaN,NaN,NaN,NaN,NaN
8,*.amltmp,NaN,NaN,NaN,NaN,NaN
9,*.amltemp,NaN,NaN,NaN,NaN,NaN


In [44]:
try:
    tab_data_set = tab_data_set.register(workspace=ws, 
                                        name='gLd',
                                        description='golddata',
                                        tags = {'format':'CSV'},
                                        create_new_version=True)
except Exception as ex:
    print(ex)

In [45]:
print("Datasets:")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name, 'version', dataset.version)

Datasets:
	 GLD_123 version 1
	 gLd version 1


In [46]:
import os

# Create a folder for the experiment files
experiment_folder = 'gld_training_from_tab_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

gld_training_from_tab_dataset folder created


In [47]:
%%writefile $experiment_folder/gld_training.py
# Import libraries
import os
import argparse
from azureml.core import Run, Dataset
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

# Get the script arguments (regularization rate and training dataset ID)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# Set regularization hyperparameter (passed as an argument to the script)
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# Get the training dataset
print("Loading Data...")
gld = run.input_datasets['GLD_123'].to_pandas_dataframe()

# Separate features and labels
X, y = gld[['SPX','USO','USO','USO']].values, gld['GLD'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = RandomForestRegressor(n_estimators=100).fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)

# calculate r2_score
r2_score=metrics.r2_score(y_hat,y_test)
run.log("r2_score",np.float(r2_score))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/gld_model.pkl')

run.complete()

Overwriting gld_training_from_tab_dataset/gld_training.py


In [48]:
gld_ds = ws.datasets.get("GLD_123")

In [49]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails


# Create a Python environment for the experiment (from a .yml file)
env = Environment.from_conda_specification("experiment_env", "environment.yml")

# Get the training dataset
gld_ds = ws.datasets.get("GLD_123")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                              script='gld_training.py',
                              arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                           '--input-data', gld_ds.as_named_input('GLD_123')], # Reference to dataset
                              environment=env,
                              docker_runtime_config=DockerConfiguration(use_docker=True),
                              compute_target=cluster_name) 

# submit the experiment
experiment_name = 'mslearn-train-gld'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'mslearn-train-gld_1690063405_f592aa94',
 'target': 'your-compute-cluster',
 'status': 'Finalizing',
 'startTimeUtc': '2023-07-22T22:03:44.705185Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '60f6eb36-ef7a-4d7e-acab-cbb63f2cde99',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '0b4f8763-bd85-4fc0-870a-a75a00a1ca27'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'GLD_123', 'mechanism': 'Direct'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'gld_training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--regularization',
   '0.1',
   '--input-data',
   'DatasetConsumptionConfig:GLD_123'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'your-compute-cluster',
  'dataReferences': {},
  'data': {'GLD_123': {'dataLocation': {'datase

In [50]:
run=experiment.get_run("")
run.download(target_directory="E:\AZURE GIT\SIMPLE GLD", target_folder="Users/divyapratapsingh")

AttributeError: 'ScriptRun' object has no attribute 'download'